In [41]:
%reset -f
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import figurefirst as fifi
import figure_functions as ff

from drone_model_body_level import DroneSimulator
from pybounds import SlidingEmpiricalObservabilityMatrix, SlidingFisherObservability, ObservabilityMatrixImage, colorline

pd.set_option('display.max_columns', None)
plt.style.use('default')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set trajectories

In [42]:
fs = 10
dt = 1 / fs
T = 2.4

In [43]:
# traj_range = {'w': (0.0, 2.0),
#               'zeta': (-np.pi, np.pi),
#               'z': (0.1, 10.0),
#               'v_x': (0.05, 2.0),
#               'v_y': (-0.2, 0.2),
#               'v_x_dot_ratio': (0.05, 2.0),
#               'v_y_dot': (-0.5, 0.5),
#               'psi': (-np.pi, np.pi),
#               'psi_dot': (1.5*(-np.pi)/T, 1.5*(np.pi)/T),
#               'psi_ddot': ((-4)/T, (4)/T)
#               }

In [44]:
traj_range = {'w': (0.0, 2.0),
              'zeta': (-np.pi, np.pi),
              'z': (0.6, 0.6),
              'v_x': (0.05, 1.0),
              'v_y': (-0.2, 0.2),
              'v_x_dot_ratio': (0.05, 2.0),
              'v_y_dot': (-0.5, 0.5),
              'psi': (-np.pi, np.pi),
              'psi_dot': (0.1*(-5*np.pi/6)/T, 0.1*(5*np.pi/6)/T),
              'psi_ddot': ((-5)/T, (5)/T)
              }

In [45]:
traj_range

{'w': (0.0, 2.0),
 'zeta': (-3.141592653589793, 3.141592653589793),
 'z': (0.6, 0.6),
 'v_x': (0.05, 1.0),
 'v_y': (-0.2, 0.2),
 'v_x_dot_ratio': (0.05, 2.0),
 'v_y_dot': (-0.5, 0.5),
 'psi': (-3.141592653589793, 3.141592653589793),
 'psi_dot': (-0.10908307824964561, 0.10908307824964561),
 'psi_ddot': (-2.0833333333333335, 2.0833333333333335)}

# Create data frame

In [46]:
n_traj = 10000

In [47]:
traj_data = {}
traj_data_names = traj_range.keys()
traj_data_df = pd.DataFrame(np.nan * np.zeros((n_traj, len(traj_data_names))), columns=traj_data_names)

np.random.seed(0)  # set seed
for c in traj_data_df:
    if isinstance(traj_range[c], set):
        traj_data_df.loc[:, c] = np.random.randint(low=tuple(traj_range[c])[0],
                                                   high=tuple(traj_range[c])[-1] + 1,
                                                   size=n_traj)
        traj_data_df[c] = traj_data_df[c].astype(int) 
    else:
        traj_data_df.loc[:, c] = np.random.uniform(low=traj_range[c][0],
                                                   high=traj_range[c][1],
                                                   size=n_traj)
    
traj_data_df.insert(loc=5, column='v_x_dot', value=traj_data_df['v_x'] * (traj_data_df['v_x_dot_ratio'] - 1) / T)

traj_data_df

,w,zeta,z,v_x,v_y,v_x_dot,v_x_dot_ratio,v_y_dot,psi,psi_dot,psi_ddot
0,1.097627,1.559914,0.6,0.770219,-0.052297,-0.112353,0.649908,0.427955,0.832356,-0.106662,-1.286984
1,1.430379,-2.009346,0.6,0.528153,-0.115470,-0.182452,0.170910,0.402937,-2.669645,-0.108697,-1.839385
2,1.205527,-0.697288,0.6,0.218166,-0.009238,0.088914,1.978131,-0.072383,1.395392,-0.075255,-0.263290
3,1.089766,-2.905344,0.6,0.840910,-0.167106,0.158451,1.452226,0.010806,-0.208211,-0.039977,0.917903
4,0.847310,-3.067528,0.6,0.540984,-0.104936,0.118850,1.527261,0.083200,1.518976,0.033127,1.351716
...,...,...,...,...,...,...,...,...,...,...,...
9995,1.100894,1.559326,0.6,0.684964,0.018532,0.220425,1.772332,0.399451,-2.271716,-0.046148,0.941338
9996,0.794303,-1.267528,0.6,0.995166,0.077324,-0.086597,0.791158,0.054135,2.168659,-0.033214,0.031335
9997,1.516859,-0.336428,0.6,0.608813,-0.106530,-0.157389,0.379558,0.262907,1.785198,0.014019,1.292220
9998,0.047575,-0.878850,0.6,0.453384,0.132370,-0.082239,0.564665,0.409548,0.749300,0.022964,0.212634


# Simulate trajectories

In [48]:
# Set time
tsim = np.arange(0, T + dt/2, step=dt)
tsim.shape

(25,)

In [49]:
# Set control parameters
mpc_horizon = 10
r_u = 1e-2

In [50]:
# Set sensors
o_sensors = ['psi', 'gamma', 'beta']

# Set states
o_states = None

# Set time-steps
window_size = 5
o_time_steps = np.arange(0, window_size, step=1)

In [51]:
observability_on = True
cnorm = mpl.colors.LogNorm(10 ** -2, 10 ** 6)

In [52]:
show_plot = False

In [ ]:
st = time.time()
traj_list = []
for n in range(0, n_traj):
    print(n, end=', ')
    
    # Get trajectory
    traj = traj_data_df.iloc[[n], :]
    
    # Set velocities
    v_x = traj['v_x'].values + traj['v_x_dot'].values*tsim
    v_y = traj['v_y'].values + traj['v_y_dot'].values*tsim

    # Set heading
    psi = traj['psi'].values + 1.0*(traj['psi_dot'].values * tsim) + 1.0*(traj['psi_ddot'].values * tsim**2)
        
    # Set elevation
    z = traj['z'].values * np.ones_like(tsim)
        
    # Wind
    w = traj['w'].values * np.ones_like(tsim)
    zeta = traj['zeta'].values * np.ones_like(tsim)
    
    # Run simulation
    try:
        simulator = DroneSimulator(dt=dt, mpc_horizon=mpc_horizon, r_u=r_u)
        simulator.update_setpoint(v_x=v_x, v_y=v_y, psi=psi, z=z, w=w, zeta=zeta)
        t_sim, x_sim, u_sim, y_sim = simulator.simulate(x0=None, mpc=True, return_full_output=True)
        
        # Set simulation data
        sim_data = pd.DataFrame(y_sim)
        sim_data.insert(loc=0, column='time', value=t_sim)
        sim_data['time'] = sim_data['time'].values - sim_data['time'].values[0]
           
        # Observability
        if observability_on:
            SEOM = SlidingEmpiricalObservabilityMatrix(simulator, t_sim, x_sim, u_sim, w=window_size, eps=1e-4)
            SFO = SlidingFisherObservability(SEOM.O_df_sliding, time=SEOM.t_sim, lam=1e-6, R=0.1,
                                             states=o_states, sensors=o_sensors, time_steps=o_time_steps, w=window_size)
            EV_aligned = SFO.get_minimum_error_variance()
            
            # Align & rename observability data
            EV_aligned_new = EV_aligned.copy()
            t_start = np.where((EV_aligned_new['time_initial'].values == 0.0))[0][0]
            t_end = np.where(np.isnan(EV_aligned_new['time_initial'].values))[0][-1] - 1
            EV_aligned_new = EV_aligned_new.iloc[t_start:t_end, :].reset_index(drop=True)
            time_end = EV_aligned_new['time_initial'].values[-1]
            EV_aligned_new.columns = ['o_' + item for item in EV_aligned_new.columns]
            
            # Combine simulation & observability data
            sim_data_new = sim_data.copy()
            t_end = np.where(sim_data_new['time'].values == time_end)[0][0] + 1
            sim_data_new = sim_data_new.iloc[0:t_end, :].reset_index(drop=True)
            sim_data_all = pd.concat([sim_data_new, EV_aligned_new], axis=1)
        else:
            sim_data_all = sim_data.copy()
        
        # Store data
        traj_list.append(sim_data_all)
        
    except Exception as e:
        print(f"\nError at iteration {n}: {e}")
        continue

    # Plot
    if show_plot:
        if observability_on:
            fig, ax = plt.subplots(1, 1, figsize=(3 * 1, 3 * 1), sharex=True, sharey=True, dpi=100)
            ff.plot_trajectory(sim_data_all['x'].values, sim_data_all['y'].values, sim_data_all['psi'].values,
                               color=sim_data_all['o_zeta'].values,
                               colornorm=cnorm,
                               colormap='inferno_r',
                               ax=ax,
                               size_radius=0.06,
                               nskip=0)
        
            fifi.mpl_functions.adjust_spines(ax, [])
        else:
            simulator.plot_trajectory(start_index=0, dpi=100)

print('elapsed time', time.time() - st)

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 

In [ ]:
# simulator.plot(name='setpoint')

In [ ]:
traj_all = pd.concat(traj_list)
traj_all

# Save

In [ ]:
save_dir = os.path.join(os.path.pardir, 'data')
save_dir

In [ ]:
save_name = 'drone_training_data_n=' + str(n_traj) + '_O=' + str(observability_on)
save_name

In [ ]:
traj_all.to_csv(path_or_buf=os.path.join(save_dir, save_name + '.csv'))

In [ ]:
traj_data_df.to_csv(path_or_buf=os.path.join(save_dir, save_name + '_labels.csv'))